# Analisi e visualizzazione dei dati del Lavoro in italia (1996-2018) (ISTAT)
Il progetto consiste nell'analisi dei dati presenti sul sito <a href="https://www.kaggle.com">Kaddle</a> e relativi alla condizione lavorativa in italia:
<a href="https://www.kaggle.com/mpwolke/cusersmarildownloadspopolazionecsv">Population by labour status - Italy</a>. <br>
La fonte dei dati è ISTAT e si tratta delle rilevazioni fatte dal 1996 al secondo trimestre 2019.<br>
Nell specifico l'attività condotta consiste in:
1. Caricamento e prima esplorazione del Dataset
2. Preparazione dei valori e data cleaning
3. Rimozione delle features ridondanti
4. Creazione di nuove feature e nuovi aggregati
5. Reshaping del Dataframe 
6. Visualizzazione interattiva con Plotly, Plotly Express e Plotly Dash

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.plotting.register_matplotlib_converters()
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.io as pio
#import geopandas as gpd

#imposto il template di default di Plotly Express
pio.templates.default = "plotly_white"

# Verifico la presenza del notebook nella cartella

#import os
#for dirname, _, filenames in os.walk('.'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))


## Analisi esplorativa del DataFrame
Carico il file "popolazione.csv" usando l'encoding corretto e il separatore ";"
Visualizzo le prime 5 righe del DataFrame 

In [3]:
df = pd.read_csv('popolazione_orig.csv', encoding='cp1252', sep=';', decimal='.')
df.head(5)

,itter107,territorio,tipo_dato_fol,tipo dato,sexistat1,sesso,eta1,classe di età,condizione_prof,condizione professionale,condizione_prof_eu,condizione professionale europea,time,seleziona periodo,value
0,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996,1996,1.716.452
1,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996-Q2,T2-1996,1.717.624
2,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996-Q1,T1-1996,1.719.434
3,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996-Q4,T4-1996,1713.29
4,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996-Q3,T3-1996,1.715.458


Verifico le features numeriche e i valori nulli:

In [4]:
df.describe()

,sexistat1
count,267234.000000
mean,4.000000
std,3.559033
min,1.000000
25%,1.000000
50%,2.000000
75%,9.000000
max,9.000000


In [5]:
df.isnull().sum()

itter107                            0
territorio                          0
tipo_dato_fol                       0
tipo dato                           0
sexistat1                           0
sesso                               0
eta1                                0
classe di età                       0
condizione_prof                     0
condizione professionale            0
condizione_prof_eu                  0
condizione professionale europea    0
time                                0
seleziona periodo                   0
value                               0
dtype: int64

Al momento l'unica feature numerica è relativa al codice Istat che rappresenta il sesso, non ci sono valori nulli.<br>
Verifico l'omgeneità rispetto alla feature "Territorio":

In [6]:
df['territorio'].value_counts()

Sicilia                               12147
Friuli-Venezia Giulia                 12147
Emilia-Romagna                        12147
Lombardia                             12147
Toscana                               12147
Provincia Autonoma Bolzano / Bozen    12147
Veneto                                12147
Puglia                                12147
Calabria                              12147
Umbria                                12147
Lazio                                 12147
Molise                                12147
Basilicata                            12147
Marche                                12147
Liguria                               12147
Sardegna                              12147
Campania                              12147
Trentino Alto Adige / Südtirol        12147
Abruzzo                               12147
Valle d'Aosta / Vallée d'Aoste        12147
Provincia Autonoma Trento             12147
Piemonte                              12147
Name: territorio, dtype: int64

Esploro la feature "value":

In [7]:
df.value

0         1.716.452
1         1.717.624
2         1.719.434
3           1713.29
4         1.715.458
            ...    
267229       51.679
267230       44.875
267231       28.639
267232       47.622
267233       51.084
Name: value, Length: 267234, dtype: object

La prima esplorazione evidenzia che la feature "Value" contiene valori disomogenei e non numeri decimali come ci si aspetterebbe (si tratta di valori espressi in migliaia), vi sono infatti valori espressi col primo punto come separatore delle migliaia e un secondo punto come separatore dei decimali.

### Esplorazione della condizione professionale
Inizio l'esplorazione delle diverse feature relative alla condizione professionale 

In [8]:
df['condizione professionale'].astype('category').cat.categories

Index(['disoccupati', 'forze lavoro', 'inattivi',
       'non cercano e non disponibili a lavorare', 'occupati', 'totale',
       'zona grigia dell'inattività'],
      dtype='object')

In [9]:
df_once = df[(df.territorio == 'Sardegna') & (df['time'] == '2012') 
             & (df['classe di età'] == '15-64 anni') 
             & (df['sesso'] == 'totale')]
df_once.head(40)

,itter107,territorio,tipo_dato_fol,tipo dato,sexistat1,sesso,eta1,classe di età,condizione_prof,condizione professionale,condizione_prof_eu,condizione professionale europea,time,seleziona periodo,value
89592,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,UNEM,disoccupati,2012,2012,106.792
107706,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,2,disoccupati,TOT,totale,2012,2012,106.792
125343,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,TOTIN,totale inattivi,2012,2012,432.788
160543,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,1_2,forze lavoro,TOT,totale,2012,2012,684.656
209975,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,TOT,totale,2012,2012,1.117.444
230765,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,LF,forze lavoro,2012,2012,684.656
232871,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3_4,inattivi,TOT,totale,2012,2012,432.788
234112,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3A_3B_3C,zona grigia dell'inattività,TOT,totale,2012,2012,121.814
243014,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3D,non cercano e non disponibili a lavorare,TOT,totale,2012,2012,310.974
245499,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,EMP,occupati,2012,2012,577.864


In [10]:
df_once = df[(df.territorio == 'Sardegna') & (df['time'] == '2012') 
           & (df['classe di età'] == '15-64 anni') 
           & (df['sesso'] == 'totale') & (df['condizione_prof_eu'] == 'TOT')]
df_once.head(40)

,itter107,territorio,tipo_dato_fol,tipo dato,sexistat1,sesso,eta1,classe di età,condizione_prof,condizione professionale,condizione_prof_eu,condizione professionale europea,time,seleziona periodo,value
107706,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,2,disoccupati,TOT,totale,2012,2012,106.792
160543,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,1_2,forze lavoro,TOT,totale,2012,2012,684.656
209975,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,TOT,totale,2012,2012,1.117.444
232871,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3_4,inattivi,TOT,totale,2012,2012,432.788
234112,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3A_3B_3C,zona grigia dell'inattività,TOT,totale,2012,2012,121.814
243014,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3D,non cercano e non disponibili a lavorare,TOT,totale,2012,2012,310.974
267080,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,1,occupati,TOT,totale,2012,2012,577.864


Dall'esplorazione appena riportata risulta:
1. è necessario eliminare tutte le osservazioni con condizione_prof_eu != 'TOT' in quanto rindondanti
2. Condizione professionale = 'totale' --> Totale della popolazione nella fascia di età selezionata
3. Condizione professionale = 'forze lavoro' --> Popolazione - Inattivi
4. Condizione professionale = 'occupati' --> Forza lavoro - disoccupati
5. Condizione professionale = ' inattivi' --> Zona Grigia + Non cercano e non sono disponibili a lavorare

## Preparazione dei valori
### Preparazione dei valori (value)
La strategia consiste nell'individuare il primo punto andando a tracciarne la posizione se la lunghezza della feature "value" è maggiore di 6, saranno infatti tracciate le posizioni del primo punto nei casi in cui il valore sia espresso nella forma:
"x.xxx.xxx","x.xxx.xx", "x.xxx.x", "xxx.xxx", "xx.xxx.x" ...
Nel caso la lunghezza della stringa sia 6 o meno i vaolri avranno la forma "xx.xxx", "x.xxx","xxx.xx", ...: in tutti questi casi la posizione del primo punto viene impostata a "0". 

In [11]:
#cerca l'indice del primo punto 
df['dot_index'] = np.where(df.value.str.len() > 6, df['value'].str.find('.'),0)
#Se la posizione del punto primo punto è 1 (per stringhe iniziali con lunghezza 
#iniziale > 6), procedo a rimuovere il primo punto. 
df['new_value'] = np.where(np.logical_and(df.dot_index < 2, df.dot_index > 0), 
                           df['value'].str.slice(stop=1) + df['value'].str.slice(start=2) ,df['value'])
df.new_value.head(20)

0     1716.452
1     1717.624
2     1719.434
3      1713.29
4     1715.458
5     1708.008
6     1711.124
7     1709.324
8     1704.614
9      1706.97
10    1699.068
11    1700.272
12     1702.26
13    1695.736
14    1698.004
15    1690.328
16    1693.467
17     1691.57
18    1686.992
19    1689.282
Name: new_value, dtype: object

Ora sovrascrivo la feature "value" con i valori ripuliti

In [12]:
df['value'] = df['new_value'].astype('float64')
df.value.describe()

count    267234.000000
mean        549.850701
std         766.433887
min           0.135000
25%          79.979000
50%         247.030000
75%         695.224000
max        8621.646000
Name: value, dtype: float64

Verifico il valore massimo della feature value che corrisponde al totale della forza lavoro della regione Lombradia nel secondo trimestre del 2019 ed è pari a 8621,646 (valore espresso in migliaia).

In [13]:
df_v = df[(df.value == df.value.values.max())]
df_v.head(20)

,itter107,territorio,tipo_dato_fol,tipo dato,sexistat1,sesso,eta1,classe di età,condizione_prof,condizione professionale,condizione_prof_eu,condizione professionale europea,time,seleziona periodo,value,dot_index,new_value
228110,ITC4,Lombardia,POP,popolazione 0 anni e più (in migliaia),9,totale,Y_GE15,15 anni e più,99,totale,TOT,totale,2019-Q2,T2-2019,8621.646,1,8621.646


In [14]:
#Trasformo la feature 'Value' in intero
df['value'] = df['value'] * 1000
df['value'] = df['value'].astype('int')

### Features categoriche
Preparo le feature che gestirò come Categoriche:

In [15]:
#Test con l'uso dei dati categorici --> ma Seaborn mostra tutte le categorie, anche quando filtrate..
#df['territorio'] = df['territorio'].astype('category')
#df['sesso'] = df['sesso'].astype('category')
df['classe di età'] = df['classe di età'].astype('category')
#df['condizione professionale'] = df['condizione professionale'].astype('category')
#df['time'] = df['time'].astype('category')
df['classe di età'].cat.categories
#df.time.cat.categories
#df['condizione professionale'].cat.categories

Index(['0-14 anni', '15 anni e più', '15-64 anni', '15-74 anni'], dtype='object')

### Preparazione della condizione professionale

In [16]:
# Elimino le osservazioni superflue
df = df[(df['condizione_prof_eu'] == 'TOT')]
df['condizione professionale'].value_counts()

totale                                      30888
inattivi                                    30888
disoccupati                                 23166
forze lavoro                                23166
occupati                                    23166
non cercano e non disponibili a lavorare    20328
zona grigia dell'inattività                 15246
Name: condizione professionale, dtype: int64

### Rimozione delle colonne non di interesse
Rimuovo le features non necessarie in quanto ridondanti (espressione della stessa informazione in standard differente) o contenti sempre lo stesso valore, quindi non rilevanti.

In [17]:
df.drop(['tipo_dato_fol', 'tipo dato', 'seleziona periodo', 'dot_index', 'new_value'], axis='columns', inplace=True)
#Elimino anche la feature sexistati1 
df.drop(['sexistat1'], axis='columns', inplace=True)
df.drop(['itter107'], axis='columns', inplace=True)

### Raggruppamento per zone d'Italia

In [18]:
def zona_italia(regione):
    if (regione == "Sardegna") or (regione == "Sicilia"):
        return 'Isole'
    elif ((regione =='Toscana') or (regione == 'Umbria') or (regione == 'Marche') or (regione == 'Lazio')):
        return 'Centro'
    elif ((regione =='Abruzzo') or (regione == 'Molise') or (regione == 'Campania') or (regione == 'Puglia') or (regione == 'Basilicata') or (regione == 'Calabria')):
        return 'Sud'
    else:
        return 'Nord'
df['zona'] = np.vectorize(zona_italia)(df['territorio'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166848 entries, 0 to 267116
Data columns (total 11 columns):
 #   Column                            Non-Null Count   Dtype   
---  ------                            --------------   -----   
 0   territorio                        166848 non-null  object  
 1   sesso                             166848 non-null  object  
 2   eta1                              166848 non-null  object  
 3   classe di età                     166848 non-null  category
 4   condizione_prof                   166848 non-null  object  
 5   condizione professionale          166848 non-null  object  
 6   condizione_prof_eu                166848 non-null  object  
 7   condizione professionale europea  166848 non-null  object  
 8   time                              166848 non-null  object  
 9   value                             166848 non-null  int32   
 10  zona                              166848 non-null  object  
dtypes: category(1), int32(1), object(9)
mem

<b>[NOTA]</b> Rimuovo le rilevazioni trimestrali

In [19]:
#dataframe privo dei dati trimestrali
df_anni = df[~df['time'].str.contains('Q')]

### Reshaping del Data Frame


In [20]:
df_pivot = df_anni.pivot(index=['territorio','zona','time', 'sesso', 'classe di età'], columns=['condizione professionale'], values='value')
df_pivot = df_pivot.fillna(0.0)
df_pivot.head()

condizione professionale                    disoccupati  forze lavoro  \
territorio zona time sesso   classe di età                              
Abruzzo    Sud  1996 femmine 0-14 anni              0.0           0.0   
                             15 anni e più      28068.0      224018.0   
                             15-64 anni         28068.0      221626.0   
                             15-74 anni         28068.0      223951.0   
                     maschi  0-14 anni              0.0           0.0   

condizione professionale                    inattivi  \
territorio zona time sesso   classe di età             
Abruzzo    Sud  1996 femmine 0-14 anni       91031.0   
                             15 anni e più  325979.0   
                             15-64 anni     196145.0   
                             15-74 anni     269588.0   
                     maschi  0-14 anni       95581.0   

condizione professionale                    non cercano e non disponibili a lavorare  \
territorio zona time sesso   classe di età                                             
Abruzzo    Sud  1996 femmine 0-14 anni                                           0.0   
                             15 anni e più                                       0.0   
                             15-64 anni                                          0.0   
                             15-74 anni                                          0.0   
                     maschi  0-14 anni                                           0.0   

condizione professionale                    occupati    totale  \
territorio zona time sesso   classe di età                       
Abruzzo    Sud  1996 femmine 0-14 anni           0.0   91031.0   
                             15 anni e più  195950.0  549996.0   
                             15-64 anni     193559.0  417771.0   
                             15-74 anni     195884.0  493539.0   
                     maschi  0-14 anni           0.0   95581.0   

condizione professionale                    zona grigia dell'inattività  
territorio zona time sesso   classe di età                               
Abruzzo    Sud  1996 femmine 0-14 anni                              0.0  
                             15 anni e più                          0.0  
                             15-64 anni                             0.0  
                             15-74 anni                             0.0  
                     maschi  0-14 anni                              0.0

### Calcolo ulteriori features relative ai tassi
Calcolo nuove Features a partire dai dati attuali: <br><b>Tasso di occupazione:</b> rapporto tra occupati e popolazione (per fascia di età e sesso); <br><b>Tasso di disoccupazione:</b> rapporto tra disoccupati e forza lavoro (per fascia di età e sesso); <br><b>Tasso di attività:</b> rapporto tra forza lavoro e popolazione (per fascia di età e sesso); <br><b>Tasso di inattività:</b> rapporto tra inattivi e popolazione (per fascia di età e sesso);<br><b>Gap Occupazionale:</b> Differenza tra tasso di occupazione e tasso di attività

In [21]:
df_pivot['tasso_occupazione'] = (df_pivot['occupati'] / df_pivot['totale']) * 100
df_pivot['tasso_disoccupazione'] = (df_pivot['disoccupati'] / df_pivot['forze lavoro']) * 100
df_pivot['tasso_attività'] = (df_pivot['forze lavoro'] / df_pivot['totale']) * 100
df_pivot['tasso_inattività'] = (df_pivot['inattivi'] / df_pivot['totale']) * 100
df_pivot['gap_occupazionale']=  df_pivot['tasso_attività'] - df_pivot['tasso_occupazione']
df_pivot = df_pivot.fillna(0.0)
df_pivot.head()

condizione professionale                    disoccupati  forze lavoro  \
territorio zona time sesso   classe di età                              
Abruzzo    Sud  1996 femmine 0-14 anni              0.0           0.0   
                             15 anni e più      28068.0      224018.0   
                             15-64 anni         28068.0      221626.0   
                             15-74 anni         28068.0      223951.0   
                     maschi  0-14 anni              0.0           0.0   

condizione professionale                    inattivi  \
territorio zona time sesso   classe di età             
Abruzzo    Sud  1996 femmine 0-14 anni       91031.0   
                             15 anni e più  325979.0   
                             15-64 anni     196145.0   
                             15-74 anni     269588.0   
                     maschi  0-14 anni       95581.0   

condizione professionale                    non cercano e non disponibili a lavorare  \
territorio zona time sesso   classe di età                                             
Abruzzo    Sud  1996 femmine 0-14 anni                                           0.0   
                             15 anni e più                                       0.0   
                             15-64 anni                                          0.0   
                             15-74 anni                                          0.0   
                     maschi  0-14 anni                                           0.0   

condizione professionale                    occupati    totale  \
territorio zona time sesso   classe di età                       
Abruzzo    Sud  1996 femmine 0-14 anni           0.0   91031.0   
                             15 anni e più  195950.0  549996.0   
                             15-64 anni     193559.0  417771.0   
                             15-74 anni     195884.0  493539.0   
                     maschi  0-14 anni           0.0   95581.0   

condizione professionale                    zona grigia dell'inattività  \
territorio zona time sesso   classe di età                                
Abruzzo    Sud  1996 femmine 0-14 anni                              0.0   
                             15 anni e più                          0.0   
                             15-64 anni                             0.0   
                             15-74 anni                             0.0   
                     maschi  0-14 anni                              0.0   

condizione professionale                    tasso_occupazione  \
territorio zona time sesso   classe di età                      
Abruzzo    Sud  1996 femmine 0-14 anni               0.000000   
                             15 anni e più          35.627532   
                             15-64 anni             46.331363   
                             15-74 anni             39.689670   
                     maschi  0-14 anni               0.000000   

condizione professionale                    tasso_disoccupazione  \
territorio zona time sesso   classe di età                         
Abruzzo    Sud  1996 femmine 0-14 anni                  0.000000   
                             15 anni e più             12.529350   
                             15-64 anni                12.664579   
                             15-74 anni                12.533099   
                     maschi  0-14 anni                  0.000000   

condizione professionale                    tasso_attività  tasso_inattività  \
territorio zona time sesso   classe di età                                     
Abruzzo    Sud  1996 femmine 0-14 anni            0.000000        100.000000   
                             15 anni e più       40.730842         59.269340   
                             15-64 anni          53.049637         46.950363   
                             15-74 anni          45.376556         54.623444   
                     maschi  0-14 anni            0.000000        100.000000

## Data Visualization
### Andamento tasso di occupazione

In [32]:
df1 = df_pivot[(((df_pivot.index.get_level_values(4) == '15-64 anni')) & ((df_pivot.index.get_level_values(3)) == 'totale'))]
fig = px.line(df1, x=df1.index.get_level_values(2), y='tasso_occupazione',
               title='Andamento Tasso di occupazione in italia (per zone)', line_group= df1.index.get_level_values(0), 
              hover_name= df1.index.get_level_values(0), color=df1.index.get_level_values(1),
              category_orders={"color": df1.sort_values(by='tasso_occupazione', ascending = False).index.get_level_values(1).to_list()},
                          labels={ # replaces default labels by column name
                              "color": "Zona d'Italia",  "x": "Anno di riferimento", "tasso_occupazione": "Tasso di Occupazione"
            },
             color_discrete_map={ # replaces default color mapping by value
                "Sud": "crimson", "Nord": "darkblue", "Isole":'Darkcyan', "Centro":'DarkGoldenrod'
            })
fig.show()

### Andamento tassi per zone (interattivo)

Questa visualizzazione viene aperta in Dash.<br>
Analogo al grafico statico precedente ma consente di navigare tra i diversi tassi, le etichette e le leggende si aggiornano in maniera omogenea rispetto alla visualizzazione.

In [24]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in df1.columns[7:]],
        value=df1.columns[7],
        clearable=False,
        style={"font-family":"arial", 'font-size': '1rem','font-style': 'italic','border':'2px blue solid'},
    ),
    
    dcc.Graph(id="time-series-chart"),
])

@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")])
def display_time_series(ticker):
    fig = px.line(df1, x=df1.index.get_level_values(2), y=ticker,
                  line_group= df1.index.get_level_values(0), 
                  hover_name= df1.index.get_level_values(0), 
                  color=df1.index.get_level_values(1),
                  category_orders={"color": df1.sort_values(by=ticker, ascending = False).index.get_level_values(1).to_list()},
                  labels={ # replaces default labels by column name
                    "color": "Zona d'Italia",  "x": "Anno di riferimento", ticker : ticker
                  },
                  color_discrete_map={ # replaces default color mapping by value
                "Sud": "crimson", "Nord": "darkblue", "Isole":'Darkcyan', "Centro":'DarkGoldenrod'})
    #fig.update_xaxes(rangeslider_visible=True)
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### Confronto tassi regionali (interattivo)

In [35]:
regioni = df1.index.get_level_values(0).unique()
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in regioni],
        value=regioni[0],
        clearable=False,
        style={"font-family":"arial", 'font-size': '1rem','font-style': 'italic','border':'2px blue solid'},
    ),
    
    dcc.Graph(id="time-series-chart"),
])

@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")])
def display_time_series(ticker):
    df3 = df1[((df1.index.get_level_values(0)) == ticker)]
    fig = go.Figure()
    # Create and style traces, dash options include 'dash', 'dot', and 'dashdot'
    fig.add_trace(go.Scatter(x=df3.index.get_level_values(2), y=df3['tasso_attività'], name = 'Attività',
                         line=dict(color='Darkcyan', width=4,
                                   dash='dashdot')))
    fig.add_trace(go.Scatter(x=df3.index.get_level_values(2), y=df3['tasso_occupazione'], name='Occupazione',
                         line=dict(color='Darkseagreen', width=4)))
    fig.add_trace(go.Scatter(x=df3.index.get_level_values(2), y=df3['tasso_inattività'], name = 'Inattività',
                         line = dict(color='orangered', width=4, dash='dot')))
    fig.add_trace(go.Scatter(x=df3.index.get_level_values(2), y=df3['tasso_disoccupazione'], name = 'Disoccupazione',
                         line=dict(color='crimson', width=4)))
    fig.add_trace(go.Scatter(x=df3.index.get_level_values(2), y=df3['gap_occupazionale'], name = 'GAP Occupazionale',
                         line=dict(color='darkgrey', width=4, dash='dashdot')))

    # Edit the layout
    fig.update_layout(title='Andamento dei Tassi per Regione',
                   xaxis_title='Anno di riferimento',
                   yaxis_title='Tasso')
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [34]:

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in df1.columns[7:]],
        value=df1.columns[7],
        clearable=False,
        style={"font-family":"arial", 'font-size': '1rem','font-style': 'italic','border':'2px blue solid'},
    ),
    
    dcc.Graph(id="time-series-chart"),
])

@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")])
def display_time_series(ticker):
    fig = px.bar(df1, x=df1.index.get_level_values(2), y=ticker,
                  line_group= df1.index.get_level_values(0), 
                  hover_name= df1.index.get_level_values(0), 
                  color=df1.index.get_level_values(1))
    fig.update_layout(title_text="Analisi dei diversi tassi in italia (per zone)",
                  title_font_size=30)
    fig.update_xaxes(title_text='Anno di riferimento', title_font_size=20)
    fig.update_yaxes(title_text='Gap occupazionale', title_font_size=20)
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [38]:
regioni = df1.index.get_level_values(0).unique()
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in regioni],
        value=regioni[0],
        clearable=False,
        style={"font-family":"arial", 'font-size': '1rem','font-style': 'italic','border':'2px blue solid'},
    ),
    
    dcc.Graph(id="time-series-chart"),
])

@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")])
def display_time_series(ticker):
    df3 = df1[((df1.index.get_level_values(0)) == ticker)]
    fig = go.Figure()
    # Create and style traces, dash options include 'dash', 'dot', and 'dashdot'
    fig.add_trace(go.Bar(x=df3.index.get_level_values(2), y=df3['tasso_attività'], name = 'Attività',
                         marker_color='Darkcyan'))
    fig.add_trace(go.Bar(x=df3.index.get_level_values(2), y=df3['tasso_occupazione'], name='Occupazione',
                         marker_color='Darkseagreen'))
    fig.add_trace(go.Bar(x=df3.index.get_level_values(2), y=df3['tasso_inattività'], name = 'Inattività',
                         marker_color='orangered'))
    fig.add_trace(go.Bar(x=df3.index.get_level_values(2), y=df3['tasso_disoccupazione'], name = 'Disoccupazione',
                         marker_color='crimson'))
    fig.add_trace(go.Bar(x=df3.index.get_level_values(2), y=df3['gap_occupazionale'], name = 'GAP Occupazionale',
                         marker_color='darkgrey'))

    # Edit the layout
    fig.update_layout(title='Andamento dei Tassi per Regione',
                   xaxis_title='Anno di riferimento',
                   yaxis_title='Tasso')
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [73]:
df2 = df1[((df1.index.get_level_values(0)) == 'Sardegna')]

fig = go.Figure()
# Create and style traces, dash options include 'dash', 'dot', and 'dashdot'
fig.add_trace(go.Scatter(x=df2.index.get_level_values(2), y=df2['tasso_attività'], name = 'Attività',
                         line=dict(color='Darkcyan', width=4,
                                   dash='dashdot')))
fig.add_trace(go.Scatter(x=df2.index.get_level_values(2), y=df2['tasso_occupazione'], name='Occupazione',
                         line=dict(color='Darkseagreen', width=4)))
fig.add_trace(go.Scatter(x=df2.index.get_level_values(2), y=df2['tasso_inattività'], name = 'Inattività',
                         line = dict(color='orangered', width=4, dash='dot')))
fig.add_trace(go.Scatter(x=df2.index.get_level_values(2), y=df2['tasso_disoccupazione'], name = 'Disoccupazione',
                         line=dict(color='crimson', width=4)))

# Edit the layout
fig.update_layout(title='Andamento dei Tassi per Regione Sardegna',
                   xaxis_title='Anno di riferimento',
                   yaxis_title='Tasso')


fig.show()


In [33]:
#df5 = df_pivot[(df_pivot.index.get_level_values(4) == '15-64 anni')]


In [53]:
df5 = df_pivot[(df_pivot.index.get_level_values(4) == '15-64 anni')]
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=df5.index.get_level_values(2),
    histnorm='percent',
    name='control', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=-4.0,
        end=3.0,
        size=0.5
    ),
    marker_color='#EB89B5',
    opacity=0.75
))
fig.add_trace(go.Histogram(
    x=x1,
    histnorm='percent',
    name='experimental',
    xbins=dict(
        start=-3.0,
        end=4,
        size=0.5
    ),
    marker_color='#330C73',
    opacity=0.75
))

fig.update_layout(
    title_text='Sampled Results', # title of plot
    xaxis_title_text='Value', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

Index(['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna',
       'Friuli-Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche',
       'Molise', 'Piemonte', 'Provincia Autonoma Bolzano / Bozen',
       'Provincia Autonoma Trento', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana',
       'Trentino Alto Adige / Südtirol', 'Umbria',
       'Valle d'Aosta / Vallée d'Aoste', 'Veneto'],
      dtype='object', name='territorio')

In [87]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in df6.columns[7:]],
        value=df6.columns[7],
        clearable=False,
    ),
    dcc.Graph(id="time-series-chart"),
])

@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")])
def display_time_series(ticker):
    fig = px.line(df6, x=df6.index.get_level_values(2), y=ticker,
                 title='Andamento Tasso di occupazione in italia (per zone)', color=df6.index.get_level_values(1))
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving 

In [42]:
fig = px.bar(df6, x=df6.index.get_level_values(1), y='totale',
             hover_data=['tasso_occupazione', 'totale'], color='tasso_occupazione',
             labels={'pop':'populazione '}, height=400)
fig.show()

In [86]:
df7 = df6.groupby(by=[df6.index.get_level_values(0), df6.index.get_level_values(2)]).mean()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in df7.columns[7:]],
        value=df6.columns[7],
        clearable=False,
    ),
    dcc.Graph(id="time-series-chart"),
])

@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")])
def display_time_series(ticker):
    fig = px.line(df7, x=df7.index.get_level_values(1), y=ticker,
                 title='Andamento Tasso di occupazione in italia (per zone)', color=df7.index.get_level_values(0))
    return fig

app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Envi

In [75]:
df8 = df_pivot[(df_pivot.index.get_level_values(4) == '15-64 anni')]
df8.head(30)

condizione professionale                      disoccupati  forze lavoro  \
zona   territorio time sesso   classe di età                              
Centro Lazio      1996 femmine 15-64 anni        151877.0      810119.0   
                       maschi  15-64 anni        105670.0     1244594.0   
                       totale  15-64 anni        257547.0     2054713.0   
                  1997 femmine 15-64 anni        147156.0      811699.0   
                       maschi  15-64 anni        101078.0     1235470.0   
                       totale  15-64 anni        248234.0     2047169.0   
                  1998 femmine 15-64 anni        141464.0      812769.0   
                       maschi  15-64 anni        106041.0     1237426.0   
                       totale  15-64 anni        247505.0     2050194.0   
                  1999 femmine 15-64 anni        145104.0      833130.0   
                       maschi  15-64 anni         99673.0     1227514.0   
                       totale  15-64 anni        244776.0     2060643.0   
                  2000 femmine 15-64 anni        140091.0      853524.0   
                       maschi  15-64 anni         93443.0     1218872.0   
                       totale  15-64 anni        233533.0     2072397.0   
                  2001 femmine 15-64 anni        129961.0      872552.0   
                       maschi  15-64 anni         89474.0     1211052.0   
                       totale  15-64 anni        219436.0     2083603.0   
                  2002 femmine 15-64 anni        116402.0      880517.0   
                       maschi  15-64 anni         62104.0     1222415.0   
                       totale  15-64 anni        178506.0     2102932.0   
                  2003 femmine 15-64 anni        117096.0      913316.0   
                       maschi  15-64 anni         85800.0     1268649.0   
                       totale  15-64 anni        202896.0     2181965.0   
                  2004 femmine 15-64 anni         96741.0      942562.0   
                       maschi  15-64 anni         80856.0     1278596.0   
                       totale  15-64 anni        177597.0     2221159.0   
                  2005 femmine 15-64 anni         91752.0      954417.0   
                       maschi  15-64 anni         81955.0     1268210.0   
                       totale  15-64 anni        173707.0     2222627.0   

condizione professionale                       inattivi  \
zona   territorio time sesso   classe di età              
Centro Lazio      1996 femmine 15-64 anni     1004722.0   
                       maschi  15-64 anni      507376.0   
                       totale  15-64 anni     1512098.0   
                  1997 femmine 15-64 anni      994438.0   
                       maschi  15-64 anni      505223.0   
                       totale  15-64 anni     1499662.0   
                  1998 femmine 15-64 anni      985688.0   
                       maschi  15-64 anni      492502.0   
                       totale  15-64 anni     1478190.0   
                  1999 femmine 15-64 anni      956469.0   
                       maschi  15-64 anni      490342.0   
                       totale  15-64 anni     1446811.0   
                  2000 femmine 15-64 anni      927296.0   
                       maschi  15-64 anni      487473.0   
                       totale  15-64 anni     1414768.0   
                  2001 femmine 15-64 anni      901792.0   
                       maschi  15-64 anni      485730.0   
                       totale  15-64 anni     1387522.0   
                  2002 femmine 15-64 anni      889948.0   
                       maschi  15-64 anni      467927.0   
                       totale  15-64 anni     1357875.0   
                  2003 femmine 15-64 anni      858694.0   
                       maschi  15-64 anni      421246.0   
                       totale  15-64 anni     1279939.0   
                  2004 femmine 15-64 anni      838570.0   
      

In [23]:
df8 = df_pivot[(df_pivot.index.get_level_values(4) == '15-64 anni')]

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df8.index.get_level_values(2), y=df8.tasso_occupazione[(df8.index.get_level_values(3)=='maschi')],
                    mode='lines',
                    name='lines', visible = "legendonly"))
fig.add_trace(go.Scatter(x=df8.index.get_level_values(2), y=df8.tasso_occupazione[(df8.index.get_level_values(3)=='femmine')],
                    mode='lines+markers',
                    name='lines+markers', visible = "legendonly"))
fig.add_trace(go.Scatter(x=df8.index.get_level_values(2), y=df8.tasso_occupazione[(df8.index.get_level_values(3)=='totale')],
                    mode='markers', name='markers', visible = "legendonly"))

fig.show()

In [74]:
df7.head()

#df_regioni = gpd.read_file(filename="reg2011_g.shp")
#df_regioni.head()

condizione professionale  disoccupati  forze lavoro   inattivi  \
zona   time                                                      
Centro 1996                 109469.00    1121652.00  728527.75   
       1997                 107672.25    1117259.75  724455.50   
       1998                 105329.75    1122822.75  711804.25   
       1999                 102620.50    1136328.75  690694.50   
       2000                  93365.25    1145036.75  674709.25   

condizione professionale  non cercano e non disponibili a lavorare  \
zona   time                                                          
Centro 1996                                                    0.0   
       1997                                                    0.0   
       1998                                                    0.0   
       1999                                                    0.0   
       2000                                                    0.0   

condizione professionale    occupati      totale  zona grigia dell'inattività  \
zona   time                                                                     
Centro 1996               1012182.75  1850179.50                          0.0   
       1997               1009587.50  1841715.00                          0.0   
       1998               1017493.25  1834627.25                          0.0   
       1999               1033708.25  1827023.75                          0.0   
       2000               1051671.50  1819745.75                          0.0   

condizione professionale  tasso_occupazione  tasso_disoccupazione  \
zona   time                                                         
Centro 1996                       56.424962              9.052484   
       1997                       56.403263              8.906960   
       1998                       57.101157              8.599548   
       1999                       58.641339              8.178010   
       2000                       59.977596              7.192054   

condizione professionale  tasso_attività  tasso_inattività  
zona   time                                                 
Centro 1996                    61.989994         38.010013  
       1997                    61.869790         38.130217  
       1998                    62.422181         37.577812  
       1999                    63.800830         36.199152  
       2000                    64.555989         35.444037